In [17]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import transforms
from torchvision.transforms import v2
from torchvision.datasets import ImageFolder
from torchvision.models import EfficientNet_V2_S_Weights, ResNet50_Weights
from tqdm.auto import tqdm
from catboost import CatBoostClassifier
from sklearn.utils import compute_class_weight
import random

In [18]:
# Set the seed for reproducibility
#seed = 42
#random.seed(seed)
#torch.manual_seed(seed)

# Device configuration
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.mps.is_available():
    # ı am using mps for test this project
    device = torch.device('mps')
else:
    device = torch.device('cpu')

# data dircetory
data_dir = "dataset/"

# Globel variables

BATCH_SIZE = 32


In [19]:
class RemoveTopPixels:
    def __init__(self, pixels_to_remove):
        self.pixels_to_remove = pixels_to_remove

    def __call__(self, img):
        # Görüntünün boyutlarını al
        width, height = img.size
        # Üst 200 pikseli kırp
        cropped_img = img.crop((0, self.pixels_to_remove, width, height))
        return cropped_img

In [20]:
transform = transforms.Compose([
    v2.Resize((224 , 224)),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Grayscale(),
    #v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the data
dataset = ImageFolder(root=data_dir, transform=transform)

labels = np.array(dataset.targets)

class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weights = torch.FloatTensor(class_weights).to(device)

train_dataset, test_dataset = train_test_split(dataset, test_size=0.33, stratify=labels)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [21]:
def auto_model_trainer(num_epochs, model, train_loader, test_loader, loss_fn, optimizer):

    best_model = {"model": model, "accuracy": 0, "epoch": 0}

    total_accuracy = 0
    model.train()
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        train_loss = 0.0

        # Training loop
        for inputs, targets in tqdm(train_loader, desc="Training Progress"):

            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Evaluation
        model.eval()  # Set model to evaluation mode
        test_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = loss_fn(outputs, targets)
                test_loss += loss.item()

                # Calculate accuracy
                _, predicted = torch.max(outputs, 1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()

        train_loss /= len(train_loader)  # Average training loss
        test_loss /= len(test_loader)  # Average test loss
        accuracy = 100 * correct / total  # Test accuracy
        total_accuracy += accuracy
        print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%")
        if accuracy > best_model["accuracy"]:
            best_model["model"] = model
            best_model["accuracy"] = accuracy
            best_model["epoch"] = epoch + 1
    print(f"Best model Accuracy: {best_model['accuracy']:.2f}%, Epoch: {best_model['epoch']}")
    return best_model["model"]

In [22]:
model_dict = {"efficientnet_v2_s": models.efficientnet_v2_s(weights=EfficientNet_V2_S_Weights.DEFAULT).to(device),
              #"mobilenet_v3_large": models.mobilenet_v3_large(pretrained=True).to(device),
              #"resnet50": models.resnet50(weights=ResNet50_Weights.DEFAULT).to(device)
              }

num_epochs = 20  # Set to 10 for testing, adjust as needed
saved_model = {}
for model_name, model in model_dict.items():
    # Get the number of input features to the classifier
    #num_ftrs = model.classifier[1].in_features

    # Replace the classifier
    #model.classifier[1] = nn.Linear(num_ftrs, 3).to(device)  # Replace 1000 with your number of classes (3)
    #loss_fn = nn.CrossEntropyLoss(weight=class_weights)
    loss_fn = nn.CrossEntropyLoss()
    #loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    #optimizer = optim.AdamW(model.parameters(), lr=1e-4)
    #optimizer = optim.SGD(model.parameters(), lr=0.01)
    #optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
    print(f"Training {model_name}")
    model = auto_model_trainer(num_epochs, model, train_loader, test_loader, loss_fn, optimizer)
    saved_model[model] = model



Training efficientnet_v2_s


Training Progress:   0%|          | 0/5 [00:00<?, ?it/s]

RuntimeError: Given groups=1, weight of size [24, 3, 3, 3], expected input[32, 1, 224, 224] to have 3 channels, but got 1 channels instead

In [16]:
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import RFE

for model_name, model in saved_model.items():

    model.eval()
    feature_extractor = nn.Sequential(*list(model.children())[:-1])
    feature_extractor.to(device)
    feature_extractor.eval()

    X_train = []
    y_train = []

    X_Test = []
    y_test = []

    with torch.no_grad():
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            features = feature_extractor(inputs)
            features = features.view(features.size(0), -1)  # Flattening
            X_train.extend(features.cpu().numpy())
            y_train.extend(labels.cpu().numpy())
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            features = feature_extractor(inputs)
            features = features.view(features.size(0), -1)  # Flattening
            X_Test.extend(features.cpu().numpy())
            y_test.extend(labels.cpu().numpy())

    X_train = np.array(X_train)
    y_train = np.array(y_train)

    X_Test = np.array(X_Test)
    y_test = np.array(y_test)

    """scaler = MinMaxScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_Test = scaler.transform(X_Test)"""

    pca = PCA(n_components=0.90)
    X_train = pca.fit_transform(X_train)
    X_Test = pca.transform(X_Test)

    """selector = SelectKBest(score_func=f_classif, k=75)  # İstenilen K sayısı
    X_train = selector.fit_transform(X_train, y_train)
    X_Test = selector.transform(X_Test)"""

    """selector = RFE(SVC(kernel="linear"), n_features_to_select=150)
    X_train = selector.fit_transform(X_train, y_train)
    X_Test = selector.transform(X_Test)"""


    clf = SVC(kernel="rbf", C=1, gamma="scale")
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_Test)
    acc = accuracy_score(y_test, y_pred)
    print(" Accuracy: ", acc)

 Accuracy:  0.7428571428571429
